In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from ROOT import geo2d
pygeo = geo2d.PyDraw()
from ROOT import larocv
import numpy as np
import cv2

import ROOT, sys
from ROOT import std
from larcv import larcv
if len(sys.argv) < 2:
   print 'Usage: python',sys.argv[0],'CONFIG_FILE [LARCV_FILE1 LARCV_FILE2 ...]'
   sys.exit(1)
proc = larcv.ProcessDriver('ProcessDriver')
sys.argv=[""]*4
sys.argv[1]="../../../larbysimageana.cfg"
sys.argv[2]="../../ssnet_numu_nopp_notunique.root"
sys.argv[3]="/Users/vgenty/Desktop/numu_8000.root"
proc.configure(sys.argv[1])

if len(sys.argv) > 1:
   flist=ROOT.std.vector('std::string')()
   for x in xrange(len(sys.argv)-3):
      flist.push_back(sys.argv[x+3])
   proc.override_input_file(flist)
ana_id = proc.process_id("LArbysImageAna")
extract_id = proc.process_id("LArbysImageExtract")
larbysimg_ana = proc.process_ptr(ana_id)
larbysimg_ana.SetInputLArbysFile(sys.argv[2])
larbysimg_extract = proc.process_ptr(extract_id)
proc.initialize()

In [ ]:
iy=-1
ix=2
for entry in xrange(100):
    if not proc.process_entry(entry): continue
    iy+=1
    if ix==iy:break

In [ ]:
xmin=ymin=999
xmax=ymax=0
timg_v=[None]*3
simg_v=[None]*3
for plane in xrange(3):
    timg=pygeo.image(larbysimg_extract.TrackImage(plane))
    simg=pygeo.image(larbysimg_extract.ShowerImage(plane))
    nz_pixels=np.where((timg+simg)>0.0)
    ymin_=np.min(nz_pixels[0])-20
    ymax_=np.max(nz_pixels[0])+20
    xmin_=np.min(nz_pixels[1])-20
    xmax_=np.max(nz_pixels[1])+20
    if xmin>xmin_:xmin=xmin_
    if xmax<xmin_:xmax=xmax_
    if ymin>ymin_:ymin=ymin_
    if ymax<ymax_:ymax=ymax_
        
    timg=np.where(timg>10,255,0.0)
    timg_v[plane]=timg.copy().astype(np.uint8)
    
    simg=np.where(simg>10,150,0.0)
    simg_v[plane]=simg.copy().astype(np.uint8)


In [ ]:
type2col=["orange","yellow","green","cyan","orange","magenta"]
fig, axarr = plt.subplots(1, 3, sharey=True,figsize=(30,10))
ends=[]
pxs=[]
for plane in xrange(3):
    ax=axarr[plane]
    ax.imshow(timg_v[plane]+simg_v[plane],interpolation="None",vmin=0,vmax=255,cmap="Greys")
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    ax.set_aspect(1/3.)
    ax.grid()
    
    iy=0
    iz=1
    for ix,vertex in enumerate(larbysimg_ana.Verticies()):
        if vertex.type!=1: continue
        iy+=1
        if iz!=iy: continue
        for particle in larbysimg_ana.Particles(ix,plane):
            ctor=np.array([[pt.x,pt.y] for pt in particle._ctor])
            ax.plot(ctor[:,0],ctor[:,1],lw=4)
            pxs.append((particle._ctor,plane))
        for track in larbysimg_ana.Tracks(ix,plane):
            endpt=track.end_pt()
            ends.append((endpt,plane))
            ax.plot(endpt.x,endpt.y,'o',color='red',markersize=10)
            
        vtx2d=vertex.vtx2d_v[plane]
        ax.plot(vtx2d.pt.x,vtx2d.pt.y,'*',color=type2col[vertex.type],markersize=30)

    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
geo=larocv.LArPlaneGeo()

In [ ]:
o1=2
o2=4

e1=ends[o1]
e2=ends[o2]
print "(",e1[0].x,e1[0].y,") @ plane ",e1[1]
print "(",e2[0].x,e2[0].y,") @ plane ",e2[1]
print "? ",geo.YZPoint(e1[0],e1[1],e2[0],e2[1])

px1=pxs[o1]
px2=pxs[o2]

img_tmp1=larocv.MaskImage(larbysimg_extract.ADCImage(px1[1]),px1[0],0,False)
img_tmp1=pygeo.image(img_tmp1).astype(np.uint8)
nzero1=cv2.findNonZero(img_tmp1)[:,0,:]
nzero1_v=geo2d.VectorArray("float")()
for nz in nzero1:
    nzero1_v.push_back(ROOT.cv.Point_("float")(nz[0],nz[1]))
plt.plot(nzero1[:,0],nzero1[:,1],'o')
plt.show()

img_tmp2=larocv.MaskImage(larbysimg_extract.ADCImage(px2[1]),px2[0],0,False)
img_tmp2=pygeo.image(img_tmp2).astype(np.uint8)
nzero2=cv2.findNonZero(img_tmp2)[:,0,:]
nzero2_v=geo2d.VectorArray("float")()
for nz in nzero2:
    nzero2_v.push_back(ROOT.cv.Point_("float")(nz[0],nz[1]))
plt.plot(nzero2[:,0],nzero2[:,1],'o')
plt.show()
print geo.Overlap(nzero1_v,px1[1],nzero2_v,px2[1],False)
print geo.Overlap(nzero2_v,px2[1],nzero1_v,px1[1],False)

In [ ]:
mkb=larocv.MatchBookKeeper()


In [ ]:
pair1=ROOT.std.vector("uint")(2)
pair1[0]=0
pair1[1]=1
pair2=ROOT.std.vector("uint")(3)
pair2[0]=0
pair2[1]=1
pair2[2]=2
pair3=ROOT.std.vector("uint")(3)
pair3[0]=1
pair3[1]=2
pair3[2]=3
mkb.Reset()
mkb.Match(pair1,0.5)
mkb.Match(pair2,0.8)
mkb.Match(pair3,0.6)
for ix,match in enumerate(mkb.GetResult()):
    print ix,") ",[i for i in match]

In [ ]:
seed_v=ROOT.std.vector("size_t")()
seed_v.push_back(2)
seed_v.push_back(2)
seed_v.push_back(2)
mkb=larocv.MatchBookKeeper()
clusters_per_plane=[None]*3
clusters_per_plane[0]=[0,1]
clusters_per_plane[1]=[2,3]
clusters_per_plane[2]=[4,5]

#two plane combinations
for ix,comb in enumerate(larocv.PlaneClusterCombinations(seed_v)):
    print "(",ix,")"

    if comb.size()==3:
        ix=clusters_per_plane[comb[0].first][comb[0].second]
        iy=clusters_per_plane[comb[1].first][comb[1].second]
        iz=clusters_per_plane[comb[2].first][comb[2].second]
        e1=ends[ix]
        e2=ends[iy]
        e3=ends[iz]
        px1=pxs[ix]
        px2=pxs[iy]
        px3=pxs[iz]
        pair=ROOT.std.vector("uint")(3)
        pair[0]=ix
        pair[1]=iy
        pair[2]=iz
        img_tmp1=larocv.MaskImage(larbysimg_extract.ADCImage(px1[1]),px1[0],0,False)
        img_tmp1=pygeo.image(img_tmp1).astype(np.uint8)
        nzero1=cv2.findNonZero(img_tmp1)[:,0,:]
        nzero1_v=geo2d.VectorArray("float")()
        for nz in nzero1:
            nzero1_v.push_back(ROOT.cv.Point_("float")(nz[0],nz[1]))

        img_tmp2=larocv.MaskImage(larbysimg_extract.ADCImage(px2[1]),px2[0],0,False)
        img_tmp2=pygeo.image(img_tmp2).astype(np.uint8)
        nzero2=cv2.findNonZero(img_tmp2)[:,0,:]
        nzero2_v=geo2d.VectorArray("float")()
        for nz in nzero2:
            nzero2_v.push_back(ROOT.cv.Point_("float")(nz[0],nz[1]))

        img_tmp3=larocv.MaskImage(larbysimg_extract.ADCImage(px3[1]),px3[0],0,False)
        img_tmp3=pygeo.image(img_tmp3).astype(np.uint8)
        nzero3=cv2.findNonZero(img_tmp3)[:,0,:]
        nzero3_v=geo2d.VectorArray("float")()
        for nz in nzero3:
            nzero3_v.push_back(ROOT.cv.Point_("float")(nz[0],nz[1]))
            
        overlap12=geo.Overlap(nzero1_v,px1[1],nzero2_v,px2[1],False)
        overlap13=geo.Overlap(nzero1_v,px1[1],nzero3_v,px3[1],False)
        overlap23=geo.Overlap(nzero2_v,px2[1],nzero3_v,px3[1],False)
        
        res=overlap12+overlap13+overlap23
        cnt=0
        if overlap12==0.0:cnt+=1
        if overlap13==0.0:cnt+=1
        if overlap23==0.0:cnt+=1
        if cnt>=2: continue
        print "o12:",overlap12,"o13:",overlap13,"o23:",overlap23
        print "Res: \t(",ix,iy,iz,")",res
        if res==0.0: continue
        mkb.Match(pair,res)
    
    
    if comb.size()==2: 
        ix=clusters_per_plane[comb[0].first][comb[0].second]
        iy=clusters_per_plane[comb[1].first][comb[1].second]
        e1=ends[ix]
        e2=ends[iy]
        px1=pxs[ix]
        px2=pxs[iy]
        pair=ROOT.std.vector("uint")(2)
        pair[0]=ix
        pair[1]=iy
        img_tmp1=larocv.MaskImage(larbysimg_extract.ADCImage(px1[1]),px1[0],0,False)
        img_tmp1=pygeo.image(img_tmp1).astype(np.uint8)
        nzero1=cv2.findNonZero(img_tmp1)[:,0,:]
        nzero1_v=geo2d.VectorArray("float")()
        for nz in nzero1:
            nzero1_v.push_back(ROOT.cv.Point_("float")(nz[0],nz[1]))

        img_tmp2=larocv.MaskImage(larbysimg_extract.ADCImage(px2[1]),px2[0],0,False)
        img_tmp2=pygeo.image(img_tmp2).astype(np.uint8)
        nzero2=cv2.findNonZero(img_tmp2)[:,0,:]
        nzero2_v=geo2d.VectorArray("float")()
        for nz in nzero2:
            nzero2_v.push_back(ROOT.cv.Point_("float")(nz[0],nz[1]))
        
        overlap=geo.Overlap(nzero1_v,px1[1],nzero2_v,px2[1],False)

        if overlap==0.0: continue
        print "Res: \t(",ix,iy,")",overlap

        mkb.Match(pair,overlap)
        
for ix,match in enumerate(mkb.GetResult()):
    print ix,") ",[i for i in match]